In [11]:
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
import numpy as np
from annoy import AnnoyIndex
import pickle

In [3]:
checkpoint_path = '/home/uddeshya.singh/Experiments/punjab_roberta_model/checkpoint_epoch_1'

def load_finetuned_model(checkpoint_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModel.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    model = model.to(device)
    print("Loading successfull!!!!")
    return model, tokenizer, device

model, tokenizer, device = load_finetuned_model(checkpoint_path)

Loading successfull!!!!


In [12]:
model

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-5): 6 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout)

In [4]:
input_pickle_path = '/home/uddeshya.singh/DataSet_triplet/combined_triplets_PUNJAB_10.pkl'

with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

triplets = data.get('triplets', [])
addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

print(f"Length of triplets: {len(triplets)}, {triplets[-1]}")
print(f"Length of addresses: {len(addresses)}, {addresses[-1]}")
print(f"Length of lat_lngs: {len(lat_lngs)}, {lat_lngs[-1]}")

Length of triplets: 762656, (282735, 282733, 282726, 3)
Length of addresses: 282737, vpo mandhali near guru nanak niwas gurudwara Shahid Bhagat Singh Nagar District
Length of lat_lngs: 282737, [31.191651, 75.83274809999998]


In [5]:
def get_embedding(model, tokenizer, device, text):
    
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().flatten()

In [15]:
embeddings = np.vstack([get_embedding(model, tokenizer, device, addr) for addr in tqdm(addresses[:10000], desc="Computing embeddings")])

Computing embeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:32<00:00, 108.08it/s]


In [16]:
dim = embeddings.shape[1]
annoy_index = AnnoyIndex(dim, 'angular')
for i, embedding in enumerate(embeddings):
    annoy_index.add_item(i, embedding)

annoy_index.build(10)


True

In [17]:
def find_nearest_neighbors(model, tokenizer, device, annoy_index, addresses, lat_lngs, query_address, num_neighbors=5):
    query_embedding = get_embedding(model, tokenizer, device, query_address)
    nearest_neighbors = annoy_index.get_nns_by_vector(query_embedding, num_neighbors, include_distances=True)
    neighbors_info = []
    for idx in nearest_neighbors[0]:
        neighbor_info = {
            "address": addresses[idx],
            "coordinates": lat_lngs[idx]
        }
        neighbors_info.append(neighbor_info)
    return neighbors_info

In [18]:
query_address = "164 f ghal kalan moga Moga"
coor = [30.8128771, 75.10901820000002]
nearest_neighbors_info = find_nearest_neighbors(model, tokenizer, device, annoy_index, addresses, lat_lngs, query_address, num_neighbors=10)
print("Nearest neighbors:")
for neighbor in nearest_neighbors_info:
    print(f"Address: {neighbor['address']}, Coordinates: {neighbor['coordinates']}")

Nearest neighbors:
Address:  164 f ghal kalan moga Moga, Coordinates: [30.8128771, 75.10901820000002]
Address:  10663 kikar dass mohalla s no 7 bathinda bus stand back sida bathinda Bathinda, Coordinates: [30.209438899999995, 74.93509240000002]
Address: h no d 15 371 main gali gurdwara baba piara singh kot khalsa po khalsa college amritsar pin 143002  AMRITSAR, Coordinates: [31.626265500000002, 74.8283881]
Address: street no 4 baba fateh singh nagar sukhchain sahib gurudwara road Phagwara, Coordinates: [31.2225117, 75.78506999999998]
Address: street no 5 baba fateh singh nagar phagwara sharki Phagwara, Coordinates: [31.2222972, 75.7854235]
Address: street no 1 baba fateh singh nagar phagwara, Coordinates: [31.2009375, 75.77620209999998]
Address: street no 1 baba fateh singh nagar Phagwara, Coordinates: [31.2009375, 75.77620209999998]
Address: h 2322 c block ranjit avenue Amritsar, Coordinates: [31.6590743, 74.8657997]
Address: hno 64 a makhan tailor wali gali niwi patti kot khalsa Amri